Importeer alle libraries die nodig zijn

In [3]:
from os import listdir
from os.path import isfile
from itertools import groupby
import re, pathlib
import pandas as pd

### Laden van bestanden
Haal alle bestanden op die verwerkt moeten worden. We gebruiken hiervoor een methode die "list comprehension" heet.
Deze techniek maakt een nieuwe lijst uit een oud lijstje waarbij bepaalde checks en/of transformaties gedaan kunnen worden.
In dit blok halen we alle elementen uit de "Current Working Directory" (Path.cwd()) en voert 2 checks uit. Namelijk:
* Is het gevonden element een bestand: `isfile(folder/f)`
* Voldoet het bestand aan de naamgeving-conventie, hierbij gebruiken we regex: `re.match(r'sub-[0-9]{2}-run[0-9]-Putamen_19.txt',f)`

In [5]:
#folder = pathlib.Path(r'C:\Users\Caspar\Documents\Prive')
folder = pathlib.Path.cwd()

files = [f for f in sorted(listdir(folder))
         if isfile(folder/f)
         and re.match(r'sub-[0-9]{2}-run[0-9]-Putamen_19.txt',f)
        ]
print(files)

['sub-01-run1-Putamen_19.txt', 'sub-01-run2-Putamen_19.txt', 'sub-01-run3-Putamen_19.txt', 'sub-02-run1-Putamen_19.txt', 'sub-02-run2-Putamen_19.txt', 'sub-02-run3-Putamen_19.txt']


### Groeperen per subject
Nu hebben we een lijst van bestanden, hier moeten we per subject, en dan per run iets uit gaan lezen, daarvoor gaan we eerst de lijst van bestanden in groepen opdelen.

We groeperen de lijst van bestanden per subject, lijkt lastig maar wat het doet is:
* groeperen op de eerste 6 karakters van een bestandsnaam dit doet de groupby functie
* dat levert twee dingen op, een key en een groep. De key is de eerste 6 karakters van een bestand (bv sub-01)
* de group is de bestandsnamen die horen bij deze key (bv. 'sub-01-run1-Putamen_19.txt', 'sub-01-run2-Putamen_19.txt', 'sub-01-run3-Putamen_19.txt')
* nu slaan we in een nieuwe lijst deze keys en bijbehorende bestanden op

Dat levert dus een lijst op met sub-lijstjes, waarbij die sublijstjes bestaan uit subject-nummer en een lijstje bestanden:

`[ ['subID', ['bestand1','bestand2','bestand3']]
   ['subID', ['bestand1','bestand2','bestand3']]
   [...]
]`

In [7]:
#testje
firstFile = files[1]
print(firstFile[0:6])
groupedFiles=[[key[4:],list(group)]
              for key,group in groupby(files , lambda s : s[0:6])
             ]
print(groupedFiles)

sub-01
[['01', ['sub-01-run1-Putamen_19.txt', 'sub-01-run2-Putamen_19.txt', 'sub-01-run3-Putamen_19.txt']], ['02', ['sub-02-run1-Putamen_19.txt', 'sub-02-run2-Putamen_19.txt', 'sub-02-run3-Putamen_19.txt']]]


### Test voor één subject
Nu we een gegroepeerde lijst hebben kunnen we per subject de data gaan ophalen. We beginnen met een test/voorbeeld van het eerste subject. De stappen die we doorlopen zijn:
1. Haal het subjectID op en de bijbehorende bestanden (regel 3-8)
2. Maak een dataframe met 2 kolommen, één met subjectID en één met de meetmomenten (regel 10-11)
3. Lees de 3 bestanden van het subject in naar een dataframe (regel 14-24), in substapjes
   1. Knip de naam van de run uit het tekstbestand (regel 16)
   2. Maak kolomnamen, data en data_sd (regel 19)
   3. Lees bestand in naar datafram (regel 21)
   4. Plak per bestand de gelezen data aan het dataframe met meetmomenten en subjectIDs, gemaakt in stap 2 (regel 24)

*Deze stap voegt niets toe aan het resultaat van het script, het is enkel een demonstratie van wat er gebeurd per subject*


In [16]:
#zet deze om naar True om output per stapje te hebben, als je dat niet wil zet je hem op False
verbose=False
firstSubject = groupedFiles[0]
if verbose: print(firstSubject)
#haal de het ID en de bestanden voor het subject op
subjectID = firstSubject[0]
subjectFiles = firstSubject[1]
if verbose: print(subjectFiles)
#maak een dataframe voor het subject
dataSub1 = pd.DataFrame([[subjectID,x] for x in range(1,1411)],columns=["SubjectID","Meetmoment"])
if verbose: print(dataSub1)

#voor elk bestand van dit subject, haal de data op
for file in subjectFiles:
    #extraheer de run uit de fileName
    run = file[7:11]
    if verbose: print(run)
    #maak de kolomnamen voor het dataframe, gebruik hiervoor run, we krijgen dan kolomnamen run1 en run1_sd, run2 en run2_sd etc
    columnNames = [run,run+"_sd"]
    #lees data van de schijf, geef het de juiste kolom namen
    data = pd.read_csv(folder/file,names=columnNames, sep=" ",header=None, usecols=range(2))
    if verbose: print(data.head())
    #voeg de kolommen toe aan het dataframe van dit subject
    dataSub1 = dataSub1.join(data)
#het resultaat:
if verbose: print(dataSub1)

### Uitvoeren voor alle subjects
Nu we een test hebben uitgewerkt gaan we alle subjects inlezen!
De eerste (buitenste) for loop gaat voor elk subject draaien, de binnenste loop is hetzelfde als de code hierboven. 

We hebben een aantal "assert" statements toegevoegd. Die controlern een aantal zaken, en geven een error als de check faalt:
* Zijn er wel 3 bestanden per subject (regel 5-6)
* Zijn er wel 1410 meetmomenten per bestand (regel 18-19)
* Zijn er wel evenveel dataframes als er subjects zijn gevonden (regel 24-25)

In [ ]:
subjectDataFrames = []
for subjectGroup in groupedFiles:
    subjectID = subjectGroup[0]
    subjectFiles = subjectGroup[1]
    assert len(subjectFiles) == 3, "Niet het juiste aantal bestanden "\
    "voor subject %r. We verwachten er 3, gevonden bestanden %r" % (subjectID,subjectFiles)
    dataSub = pd.DataFrame([[subjectID,x] for x in range(1,1411)],columns=["SubjectID","Meetmoment"])
    for file in subjectFiles:
        run = file[7:11]
        columnNames = [run,run+"_sd"]
        data = pd.read_csv(folder/file,names=columnNames, sep=" ",header=None, usecols=range(2))
        assert len(data.index) == 1410, "Niet het juiste aantal rijen "\
        "in %r, we vonden er %r maar verwachten er 1410" % (file,len(data.index))
        dataSub = dataSub.join(data)
    subjectDataFrames.append(dataSub)

assert len(subjectDataFrames) == len(groupedFiles), "Het inlezen van bestanden is niet goed gegegaan, we hebben "\
"niet hetzelfde aantal dataframes (%r) als groepen bestanden (%r)" % (len(subjectDataFrames),len(groupedFiles))


### Schrijven van data
Aan het einde plakken we alles aan elkaar, en schrijven we het naar een csv bestand. Hier kan nog gekozen worden om de seperator aan te passen.

In [15]:
#Plak alles aan elkaar
allData = pd.concat(subjectDataFrames,sort=False)
#hernoem kolommen
newColumnNames = {
    "run1" : "water",
    "run2" : "suiker",
    "run3" : "vet",
    "run1_sd" : "water_sd",
    "run2_sd" : "suiker_sd",
    "run3_sd" : "vet_sd"
}
allData = allData.rename(columns=newColumnNames)
#schrijf data naar bestand alles.csv
allData.to_csv(folder/"alles.csv", index=False, sep=';')